In [1]:
# import sys
# from os.path import abspath,join
# sys.path.append(abspath(join('..', 'module')))

import numpy as np
import pandas as pd
from statistics import mean

In [2]:
data = pd.read_csv('../data/univ_sbm_with_kode_database.csv')
data.facultydepartmentid.fillna(0, inplace=True)
data.accreditation.fillna('D', inplace=True)
data.rataan = data.rataan.str.replace(',','.')
data.s_baku = data.s_baku.str.replace(',','.')
data['max'] = data['max'].str.replace(',','.')
data['min'] = data['min'].str.replace(',','.')
data.rataan = data.rataan.astype('float')
data.s_baku = data.s_baku.astype('float')
data['max'] = data['max'].astype('float')
data['min'] = data['min'].astype('float')
data.facultydepartmentid = data.facultydepartmentid.astype('int')

keketatan = []
for index, row in data.iterrows():
    if row['peminat_2019'] != 0:
        ketat = (row['daya_tampung_2020']/row['peminat_2019'])*100
        keketatan.append(ketat)
    else:
        keketatan.append(0)
data['persentase_keketatan'] = keketatan


In [3]:
rataan_baru = []
s_baku_baru = []
for index, row in data.iterrows():
    if np.isnan(row['rataan']):
        try:
            jurusan = row['prodi_database']
            akreditasi = row['accreditation']
            keketatan = row['persentase_keketatan']
            tabel = data.loc[(data['prodi_database']==str(''.join(jurusan.split()))) & (data['accreditation']==str(akreditasi))]
            not_null = tabel[pd.notnull(tabel['rataan'])]
            mean_rataan = mean(not_null['rataan'].tolist())
            mean_s_baku = mean(not_null['s_baku'].tolist())
            mean_keketatan = mean(not_null['persentase_keketatan'].tolist())
            if keketatan > 100:
                rataan = float('NaN')
                s_baku = float('NaN')
            else:
                rataan = ((100-keketatan)/(100-mean_keketatan))*mean_rataan
                s_baku = ((100-keketatan)/(100-mean_keketatan))*mean_s_baku
        except Exception as e:
            rataan = float('NaN')
            s_baku = float('NaN')
    else:
        rataan = row['rataan']
        s_baku = row['s_baku']
    
    rataan_baru.append(rataan)
    s_baku_baru.append(s_baku)
    
data['rataan_baru'] = rataan_baru
data['s_baku_baru'] = s_baku_baru

In [6]:
len(data) - len(data.loc[data.rataan_baru.isnull()])

1996

In [9]:
data.to_csv('../data/univ_sbm_with_rataan.csv',index=False)